In [1]:
from collections import defaultdict
from datetime import datetime, timedelta
import os
from copy import deepcopy
import yaml

from clickhouse_driver import Client

#from src.cg import CoinGeckoAPI
#from src.handle_portfolio import get_coin_date_out, get_coin_date_enter, get_date_lst_coins
#from src.get_data import get_current_price_vol_cg, GetRawData, query_get_coins_volume_per_day, get_ndau_cur_day
#from src.calc_metrics import (
#    percentage_wrt_portfolio, postprocess_metric, iterator_window_df,
#    roi_enter, get_corr, get_beta, history_diff_trend
#)
from google_sheet import write_df, write_update_time, write_data_to_gs_no_resize
from tqdm.notebook import tqdm

from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import pygsheets
import scipy
from scipy import stats



from datetime import datetime, timedelta
import time

In [2]:
dt_calc = '2023-05-14'

In [3]:
def get_config(
        path_dir_main_config: str,
        path_sensitive_config: str
):
    path_main_config = os.path.join(path_dir_main_config, 'config.yaml')
    path_input_csv = os.path.join(path_dir_main_config, 'input.csv')
    
    with open(path_main_config,'r') as stream:
        main_config = yaml.safe_load(stream)

    with open(path_sensitive_config,'r') as stream:
        conf = yaml.safe_load(stream)
        cli_creds = conf['clickhouse']
        cg_api_key = conf['cg-api-key']
    
    input_p = pd.read_csv(path_input_csv)
    input_p = input_p.set_index('coingecko_id')
    return input_p, main_config, cli_creds, cg_api_key


def setup_logging():
    file_path = os.path.dirname(__file__)
    logging.basicConfig(
        filename=os.path.join(file_path, "main.log"),
        filemode='a',
        level=logging.INFO,
        format='%(asctime)s %(levelname)-8s %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    
def get_enter_prices(input_c):
    buy_prices_usdt = input_p['price_USDT'].to_dict()
    buy_prices_btc = input_p['price_BTC'].to_dict()
    buy_prices_eth = input_p['price_ETH'].to_dict()
    return buy_prices_usdt, buy_prices_btc, buy_prices_eth

def get_info(input_c):
    info = input_p[['ticker', 'price_USDT', 'price_BTC', 'price_ETH', 'amount_usdt', 'dt']] #.set_index('ticker')
    return info

In [4]:
input_p, main_config, cli_creds, cg_api_key = get_config('configs/config_5_25_03_2023/', 'configs/sensitive.yaml')

cli = Client(
    host=main_config['clickhouse']['client']['host'],
    user=cli_creds['user'],
    password=cli_creds['password']
        #settings=main_config['clickhouse']['client']['settings']
    )

cli.execute('select now();')


def init_():
    portfolio_members = pd.DataFrame(
        columns = [
            'dt',
            'coins'
        ]

    )

    unrealized = pd.DataFrame(
        columns = [
            'coingecko_id', 'dt_in', 
            'price_in_usd', 'price_in_btc', 'price_in_eth',
            'usd_in', 'btc_in', 'eth_in'

        ]
    )

    realized = pd.DataFrame(
        columns = [
            'coingecko_id', 'dt_in', 'dt_out',
            'price_in_usd', 'price_in_btc', 'price_in_eth',
            'usd_in', 'btc_in', 'eth_in',

            'price_out_usd', 'price_out_btc', 'price_out_eth',
            'usd_out', 'btc_out', 'eth_out',

            'fixed_usd', 'fixed_btc', 'fixed_eth',

        ]
    )
    
    return portfolio_members, unrealized, realized

In [5]:
actual_coins = cli.query_dataframe('select * from DEV_ODS_CROSS_BC_RESOURCES.SELECTED_FOR_DASH_TOKENS_V')
coins_today = actual_coins['coingecko_id'].values.tolist()
#coins_today.remove('arbitrum')
coins_today.remove('ethereum')
coins_today.remove('bitcoin')

In [6]:
coins_today.remove('pip')

In [7]:
btc_sum = cli.query_dataframe(
    '''
        SELECT *
        FROM DEV_DM_ANALYTCS.EMA200_BTC_SUMMARY
        WHERE 
            coingecko_id in {}
        ORDER BY tech_load_ts asc
    '''.format(
        coins_today
    )
)

btc_sum['dt'] = pd.to_datetime(btc_sum['tech_load_ts']).dt.date - timedelta(days=1) #
btc_sum = btc_sum.groupby(['coingecko_id', 'dt']).first().reset_index()
btc_sum['dt'] = btc_sum['dt'].astype(str)

btc_sum = btc_sum[btc_sum['dt'] >= '2023-03-24']


In [8]:
btc_sum.loc[(
    btc_sum['coingecko_id'].isin(['arbitrum', 'injective-protocol'])
    & (btc_sum['dt'] < '2023-04-16')
), ['days_above_200EMA', 'days_above_200EMA_dt_candle_close']]  = -1

btc_sum.loc[(
    btc_sum['coingecko_id'].isin(['gamma-strategies', 'pip'])
    & (btc_sum['dt'] < '2023-04-18')
), ['days_above_200EMA', 'days_above_200EMA_dt_candle_close']]  = -1

btc_sum.loc[(
    btc_sum['coingecko_id'].isin(['havven'])
    & (btc_sum['dt'] < '2023-04-19')
), ['days_above_200EMA', 'days_above_200EMA_dt_candle_close']]  = -1



In [9]:

def generate_dt_range(first_dt, last_dt):
    idx_dt = pd.date_range(first_dt, last_dt)
    return idx_dt

dt_range = generate_dt_range('2023-03-24', '2023-04-24')

lst_df = []
for c, df in btc_sum.groupby('coingecko_id'):
    df = df.set_index(pd.to_datetime(df['dt']))
    df = df.reindex(dt_range)
    df = df.fillna(method='ffill')
    lst_df.append(df.reset_index())
btc_sum = pd.concat(lst_df, axis=0)
btc_sum['dt'] = btc_sum['index'].astype(str)
btc_sum = btc_sum.drop(columns=['index'])
btc_sum.loc[btc_sum['coingecko_id'] == 'kwenta', 'days_above_200EMA'] = 2

btc_sum_input = btc_sum[btc_sum['dt'] == '2023-03-25']
btc_sum = btc_sum[btc_sum['dt'] > '2023-03-25']

#get_input
btc_sum_input[btc_sum_input['days_above_200EMA'] > 0]['coingecko_id'].values

array(['3xcalibur', 'camelot-token', 'floki', 'gains-network', 'gmx',
       'hamachi-finance', 'hop-protocol', 'immutable-x', 'joe', 'kaspa',
       'kwenta', 'magic', 'multichain', 'nakamoto-games', 'oath', 'okb',
       'phoenix-global', 'plutusdao', 'radiant-capital', 'ssv-network',
       'velodrome-finance'], dtype=object)

In [10]:
btc_sum = btc_sum.reset_index(drop=True)
btc_sum.loc[1441] = ['winr-protocol', '2023-03-27', '1h', 0.724942, 0.724942, 1, 1.0, '2023-03-28 03:06:06', 0, '1970-01-01']
btc_sum['dt_candle_close'] = pd.to_datetime(btc_sum['dt_candle_close']).dt.date.astype(str)

btc_sum.loc[
    btc_sum['dt_candle_close'] != '1970-01-01',
    'days_above_200EMA'
] = btc_sum.loc[
    btc_sum['dt_candle_close'] != '1970-01-01',
    'days_above_200EMA_dt_candle_close'
]

In [11]:
new_btc_sum = cli.query_dataframe(
    '''
        SELECT *
        FROM DM_EMA_PRICES.EMA200_BTC_SUMMARY
        WHERE 
            coingecko_id in {}
        ORDER BY tech_load_ts asc
    '''.format(
        coins_today
    )
)

new_btc_sum['dt'] = pd.to_datetime(new_btc_sum['tech_load_ts']).dt.date - timedelta(days=1) #
new_btc_sum = new_btc_sum.groupby(['coingecko_id', 'dt']).first().reset_index()
new_btc_sum['dt'] = new_btc_sum['dt'].astype(str)
n1 = new_btc_sum[['coingecko_id', 'dt', 'days_above_200EMA_all_candles']]

n2 = btc_sum[['coingecko_id', 'dt', 'days_above_200EMA']]
n1.columns = ['coingecko_id', 'dt', 'days_above_200EMA']
btc_sum = pd.concat([n1, n2])

btc_sum.sort_values(by='dt')['dt'].unique()

array(['2023-03-26', '2023-03-27', '2023-03-28', '2023-03-29',
       '2023-03-30', '2023-03-31', '2023-04-01', '2023-04-02',
       '2023-04-03', '2023-04-04', '2023-04-05', '2023-04-06',
       '2023-04-07', '2023-04-08', '2023-04-09', '2023-04-10',
       '2023-04-11', '2023-04-12', '2023-04-13', '2023-04-14',
       '2023-04-15', '2023-04-16', '2023-04-17', '2023-04-18',
       '2023-04-19', '2023-04-20', '2023-04-21', '2023-04-22',
       '2023-04-23', '2023-04-24', '2023-04-25', '2023-04-26',
       '2023-04-27', '2023-04-28', '2023-04-29', '2023-04-30',
       '2023-05-01', '2023-05-02', '2023-05-03', '2023-05-04',
       '2023-05-05', '2023-05-06', '2023-05-07', '2023-05-08',
       '2023-05-09', '2023-05-10', '2023-05-11', '2023-05-12',
       '2023-05-13', '2023-05-14', '2023-05-16', '2023-05-17',
       '2023-05-18', '2023-05-19', '2023-05-20', '2023-05-21',
       '2023-05-22', '2023-05-23', '2023-05-24'], dtype=object)

In [12]:

all_action_coins = coins_today #list(set(coins_this_day).union(prev_coins))
prices = cli.query_dataframe(
    '''
    SELECT 
    coingecko_id, price, date_of_candle_close as dt_candle_close
    FROM DEV_ODS_CG_MARKET.DEV_ALL_TOKEN_METRICS 
    where coingecko_id in {lst_coins}    
    '''.format(lst_coins = all_action_coins + ['bitcoin', 'ethereum'])

)
prices['dt_candle_close'] = prices['dt_candle_close'].astype(str)
prices = prices.drop_duplicates(subset=['coingecko_id', 'dt_candle_close'])

prices = prices[prices['dt_candle_close'] >= '2023-03-25']

prices[prices['coingecko_id'] == 'optimism']

,coingecko_id,price,dt_candle_close
76,optimism,1.669223,2023-05-23
77,optimism,1.595588,2023-05-24
22398,optimism,2.191602,2023-03-25
22399,optimism,2.269058,2023-03-26
22400,optimism,2.071993,2023-03-27
...,...,...,...
22452,optimism,1.739786,2023-05-18
22453,optimism,1.668712,2023-05-19
22454,optimism,1.689870,2023-05-20
22455,optimism,1.636471,2023-05-21


In [13]:
#ReDo input
def input_(dt_input, first_day_coins, input_p, portfolio_members, unrealized, amount=10_000):
    all_coins = first_day_coins
    
    portfolio_members.loc[dt_input] = [dt_input, all_coins]
    
    print(all_coins)
    
    for cg_id in all_coins:
        unrealized.loc['{} {}'.format(cg_id, dt_input)] = {
            'coingecko_id': cg_id,
            'dt_in': dt_input,
            'price_in_usd': input_p.loc[cg_id, 'price_USDT'],
            'price_in_btc': input_p.loc[cg_id, 'price_BTC'],
            'price_in_eth': input_p.loc[cg_id, 'price_ETH'],
            
            'usd_in': amount,
            'btc_in': amount,
            'eth_in': amount
            
            
        }
        
    return unrealized, portfolio_members

fi_coins = btc_sum_input[btc_sum_input['days_above_200EMA'] > 0]['coingecko_id'].values.tolist()


portfolio_members, unrealized, realized = init_()
input_('2023-03-25', fi_coins, input_p, portfolio_members, unrealized);





['3xcalibur', 'camelot-token', 'floki', 'gains-network', 'gmx', 'hamachi-finance', 'hop-protocol', 'immutable-x', 'joe', 'kaspa', 'kwenta', 'magic', 'multichain', 'nakamoto-games', 'oath', 'okb', 'phoenix-global', 'plutusdao', 'radiant-capital', 'ssv-network', 'velodrome-finance']


In [14]:
def price_wrt_base_coin(price_ts, base_coin):
    price_ts_wrt = price_ts.merge(
        (price_ts.loc[price_ts['coingecko_id'] == base_coin, ['price', 'dt_candle_close']]
         .rename(columns={'price': f'price_{base_coin}'})
         ),
        on='dt_candle_close'
    )
    price_ts_wrt[f'price_wrt_{base_coin}'] = price_ts_wrt['price'] / price_ts_wrt[f'price_{base_coin}']

    return price_ts_wrt

In [15]:
prices_wrt_btc =  price_wrt_base_coin(prices, 'bitcoin')[['coingecko_id', 'dt_candle_close', 'price_wrt_bitcoin']]
price_wrt_eth =  price_wrt_base_coin(prices, 'ethereum')[['coingecko_id', 'dt_candle_close', 'price_wrt_ethereum']]
prices_wrt_usd = prices[['coingecko_id', 'dt_candle_close', 'price']].rename(columns={"price": 'price_wrt_usd'})


prices = pd.merge(
    pd.merge(
        prices_wrt_btc, price_wrt_eth, on=['coingecko_id','dt_candle_close']
    ), 
    prices_wrt_usd, on=['coingecko_id','dt_candle_close' ]
)
    

In [16]:
dict_above_ema = {}
for dt, df_dt in btc_sum.groupby('dt'):
    dict_above_ema[dt] = df_dt.loc[df_dt['days_above_200EMA'] > 0, 'coingecko_id'].values.tolist()
    

In [68]:
def coins_in(row):
    return [coin for coin in row['coins_in_portolio'] if coin not in row['yesterday']]

def coins_out(row):
    return [coin for coin in row['yesterday'] if coin not in row['coins_in_portolio']]

portfolio_coins = pd.DataFrame(columns=['dt_candle_close', 'coins_in_portolio'])
for dt, coins in dict_above_ema.items():
    portfolio_coins.loc[dt] = [dt, coins]

portfolio_coins['yesterday'] = portfolio_coins['coins_in_portolio'].shift(1).fillna("").apply(list)
portfolio_coins['coins_in'] = [[] for i in range(len(portfolio_coins))]
portfolio_coins['coins_out'] = [[] for i in range(len(portfolio_coins))]
for i, row in portfolio_coins.iterrows():
    row['coins_in'] = coins_in(row)
    row['coins_out'] = coins_out(row)

In [69]:
portfolio_coins = portfolio_coins.drop(columns=['yesterday'])
portfolio_coins['tech_load_ts'] = datetime.today()
portfolio_coins['dt_candle_close'] = pd.to_datetime(portfolio_coins['dt_candle_close'])
    
cli.execute('INSERT INTO PORTFOLIO_DYNAMIC_0.PORTFOLIO_COINS VALUES;', portfolio_coins.to_dict('records'))

59

In [70]:
def handle_out(dt_out, coins_out, unrealized, realized, prices):
    for coin_out in coins_out:  
        if coin_out == 'optimism':
            print('out', dt_out)
        prices_in = unrealized.loc[(unrealized['coingecko_id'] == coin_out)].squeeze()
        
        dt_in = prices_in.loc['dt_in']
        price_in_usd = prices_in.loc['price_in_usd']
        price_in_btc = prices_in.loc['price_in_btc']
        price_in_eth = prices_in.loc['price_in_eth']
        usd_in = prices_in.loc['usd_in']
        btc_in = prices_in.loc['btc_in']
        eth_in = prices_in.loc['eth_in']

        prices_out = prices[
            (prices['coingecko_id'] == coin_out) & (prices['dt_candle_close'] == dt_out)
        ].squeeze()
        
        

        price_out_usd = prices_out.loc['price_wrt_usd']
        price_out_btc = prices_out.loc['price_wrt_bitcoin']
        price_out_eth = prices_out.loc['price_wrt_ethereum']
        usd_out = usd_in / price_in_usd * price_out_usd
        btc_out = btc_in / price_in_btc * price_out_btc
        eth_out = eth_in / price_in_eth * price_out_eth
        
        fixed_usd = usd_out - usd_in
        fixed_btc = btc_out - btc_in
        fixed_eth = eth_out - eth_in
        
        

        realized.loc['{} {} {}'.format(coin_out, dt_in, dt_out)] = {
            'coingecko_id': coin_out,
            'dt_in': dt_in,
            'dt_out': dt_out,
            'price_in_usd': price_in_usd,
            'price_in_btc': price_in_btc,
            'price_in_eth': price_in_eth,
            'usd_in': usd_in,
            'btc_in': btc_in,
            'eth_in': eth_in,
    
            'price_out_usd': price_out_usd,
            'price_out_btc': price_out_btc,
            'price_out_eth': price_out_eth,
            'usd_out': usd_out,
            'btc_out': btc_out,
            'eth_out': eth_out,
            
            'fixed_usd': fixed_usd,
            'fixed_btc': fixed_btc,
            'fixed_eth': fixed_eth,
        }
        
        unrealized = unrealized.drop('{} {}'.format(coin_out, dt_in))
    
    return unrealized, realized

def handle_in(dt_calc, coins_in, unrealized, prices, amount=10_000):
    dt_in = dt_calc
    for coin_in in coins_in:
        prices_in = prices[
                (prices['coingecko_id'] == coin_in) & (prices['dt_candle_close'] == dt_in)
            ].squeeze()
        
        unrealized.loc['{} {}'.format(coin_in, dt_in)] = {
            'coingecko_id': coin_in,
            'dt_in': dt_in, 
            'price_in_usd': prices_in.loc['price_wrt_usd'], 
            'price_in_btc': prices_in.loc['price_wrt_bitcoin'], 
            'price_in_eth': prices_in.loc['price_wrt_ethereum'], 
            
            'usd_in': amount,
            'btc_in': amount,
            'eth_in': amount
        }
        
    return unrealized

def make_snapshot(dt_calc, unrealized, prices):
    pass

def step_day(dt_calc, coins_this_day, portfolio_members, unrealized, realized, prices):
    
    prev_coins = portfolio_members.iloc[-1]['coins']

    coins_out = [coin for coin in prev_coins if coin not in coins_this_day]
    coins_in =  [coin for coin in coins_this_day if coin not in prev_coins]
    
    unrealized, realized = handle_out(dt_calc, coins_out, unrealized, realized, prices)
    
    snapshot = pd.DataFrame(columns=realized.columns)
    unrealized_before = unrealized[unrealized['dt_in'] < dt_calc]
    coins = unrealized_before['coingecko_id'].values.tolist()
    _, snapshot = handle_out(dt_calc, coins, unrealized_before, snapshot, prices)

    unrealized = handle_in(dt_calc, coins_in, unrealized, prices)
    portfolio_members.loc[dt_calc] = [dt_calc, coins_this_day]
    
    
    
    return portfolio_members, unrealized, realized, snapshot

In [71]:
portfolio_members, unrealized, realized = init_()
input_('2023-03-25', fi_coins, input_p, portfolio_members, unrealized);

['3xcalibur', 'camelot-token', 'floki', 'gains-network', 'gmx', 'hamachi-finance', 'hop-protocol', 'immutable-x', 'joe', 'kaspa', 'kwenta', 'magic', 'multichain', 'nakamoto-games', 'oath', 'okb', 'phoenix-global', 'plutusdao', 'radiant-capital', 'ssv-network', 'velodrome-finance']


In [72]:
lst_snapshots = []
lst_realized = []
lst_unrealized = []
dts =  [
    '2023-03-26', '2023-03-27', '2023-03-28', '2023-03-29', '2023-03-30',
    '2023-03-31', '2023-04-01', '2023-04-02', '2023-04-03', '2023-04-04',
    '2023-04-05', '2023-04-06', '2023-04-07', '2023-04-08', '2023-04-09',
    '2023-04-10', '2023-04-11', '2023-04-12', '2023-04-13', '2023-04-14',
    '2023-04-15', '2023-04-16', '2023-04-17', '2023-04-18', '2023-04-19',
    '2023-04-20', '2023-04-21', '2023-04-22', '2023-04-23', '2023-04-24',
    '2023-04-25', '2023-04-26', '2023-04-27', '2023-04-28', '2023-04-29',
    '2023-04-30', '2023-05-01', '2023-05-02', '2023-05-03', '2023-05-04', 
    '2023-05-05', '2023-05-06', '2023-05-07', '2023-05-08', '2023-05-09',
    '2023-05-10', '2023-05-11', '2023-05-12', '2023-05-13', '2023-05-14',
    '2023-05-16', '2023-05-17', '2023-05-18', '2023-05-19', '2023-05-20', 
    '2023-05-21', '2023-05-22', '2023-05-23', '2023-05-24'
]
for dt_calc in dts:
    print(dt_calc)
    portfolio_members, unrealized, realized, snapshot = (
        step_day(dt_calc, dict_above_ema[dt_calc], portfolio_members.copy(), unrealized.copy(), realized.copy(), prices)
    )
    
    lst_snapshots.append(snapshot)
    lst_realized.append(realized)
    lst_unrealized.append(unrealized)
    

snapshots = pd.concat(lst_snapshots)
#realized['roi_btc'] = (realized['btc_out'] - realized['btc_in']) / realized['btc_in'] * 100

2023-03-26
2023-03-27
out 2023-03-27
2023-03-28
2023-03-29
out 2023-03-29
2023-03-30
2023-03-31
out 2023-03-31
2023-04-01
out 2023-04-01
2023-04-02
out 2023-04-02
2023-04-03
2023-04-04
2023-04-05
2023-04-06
out 2023-04-06
2023-04-07
out 2023-04-07
2023-04-08
out 2023-04-08
2023-04-09
out 2023-04-09
2023-04-10
out 2023-04-10
2023-04-11
out 2023-04-11
2023-04-12
2023-04-13
2023-04-14
out 2023-04-14
2023-04-15
out 2023-04-15
2023-04-16
out 2023-04-16
2023-04-17
out 2023-04-17
2023-04-18
out 2023-04-18
2023-04-19
out 2023-04-19
2023-04-20
out 2023-04-20
2023-04-21
out 2023-04-21
2023-04-22
out 2023-04-22
2023-04-23
out 2023-04-23
2023-04-24
2023-04-25
out 2023-04-25
2023-04-26
out 2023-04-26
2023-04-27
2023-04-28
2023-04-29
2023-04-30
2023-05-01
2023-05-02
2023-05-03
2023-05-04
2023-05-05
2023-05-06
2023-05-07
2023-05-08
2023-05-09
2023-05-10
2023-05-11
2023-05-12
2023-05-13
2023-05-14
2023-05-16
2023-05-17
2023-05-18
2023-05-19
2023-05-20
2023-05-21
2023-05-22
2023-05-23
2023-05-24


In [77]:
#for realized, dt_c in zip(lst_realized, dts):

realized['tech_load_ts'] = datetime.today()
realized['dt_in'] = pd.to_datetime(realized['dt_in'])
realized['dt_out'] = pd.to_datetime(realized['dt_out'])
realized['dt_candle_close'] = realized['dt_out']
num = cli.execute('INSERT INTO PORTFOLIO_DYNAMIC_0.REALIZED_HISTORY VALUES;', realized.to_dict('records'))
print(num)

79


In [76]:
print('\n'.join(unrealized.columns.tolist()))
for unrealized, dt_c in zip(lst_unrealized, dts):
    unrealized['tech_load_ts'] = datetime.today()
    unrealized['dt_candle_close'] = datetime.strptime(dt_c, '%Y-%m-%d')
    unrealized['dt_in'] = pd.to_datetime(unrealized['dt_in'])
    #unrealized['dt_out'] = pd.to_datetime(unrealized['dt_out'])
    num = cli.execute('INSERT INTO PORTFOLIO_DYNAMIC_0.UNREALIZED_HISTORY VALUES;', unrealized.to_dict('records'))
    print(num)

coingecko_id
dt_in
price_in_usd
price_in_btc
price_in_eth
usd_in
btc_in
eth_in
23
19
23
21
23
23
23
20
20
22
25
22
20
20
16
22
16
14
21
22
22
22
22
22
22
16
16
18
13
15
17
14
15
10
10
10
8
10
9
8
11
7
8
7
7
7
7
7
7
7
7
9
9
9
9
8
8
8
9


In [80]:
#print('\n'.join(snapshots.columns.tolist()))
for snapshots, dt_c in zip(lst_snapshots, dts):
    snapshots['tech_load_ts'] = datetime.today()
    snapshots['dt_candle_close'] = datetime.strptime(dt_c, '%Y-%m-%d')
    snapshots['dt_in'] = pd.to_datetime(snapshots['dt_in'])
    snapshots['dt_out'] = pd.to_datetime(snapshots['dt_out'])
    num = cli.execute('INSERT INTO PORTFOLIO_DYNAMIC_0.SNAPSHOTS VALUES;', snapshots.to_dict('records'))
    print(num)

20
18
19
21
21
22
22
20
18
20
22
22
19
19
15
15
15
14
14
19
22
22
22
22
22
13
16
15
12
13
15
13
13
10
10
9
8
8
9
8
8
7
7
7
7
7
7
7
7
7
7
7
9
8
9
8
7
8
7


In [79]:
lst_snapshots[-1]

,coingecko_id,dt_in,dt_out,price_in_usd,price_in_btc,price_in_eth,usd_in,btc_in,eth_in,price_out_usd,price_out_btc,price_out_eth,usd_out,btc_out,eth_out,fixed_usd,fixed_btc,fixed_eth
radiant-capital 2023-03-25 2023-05-24,radiant-capital,2023-03-25,2023-05-24,0.339843,1.228191e-05,1.937551e-04,10000,10000,10000,0.295412,1.121578e-05,1.640505e-04,8692.593559,9131.943664,8466.903578,-1307.406441,-868.056336,-1533.096422
okb 2023-04-14 2023-05-24,okb,2023-04-14,2023-05-24,49.850965,1.636153e-03,2.370530e-02,10000,10000,10000,44.980967,1.707774e-03,2.497921e-02,9023.088444,10437.741727,10537.396565,-976.911556,437.741727,537.396565
gamma-strategies 2023-04-20 2023-05-24,gamma-strategies,2023-04-20,2023-05-24,0.314238,1.112126e-05,1.616919e-04,10000,10000,10000,0.193609,7.350671e-06,1.075166e-04,6161.225566,6609.564848,6649.472785,-3838.774434,-3390.435152,-3350.527215
injective-protocol 2023-04-20 2023-05-24,injective-protocol,2023-04-20,2023-05-24,7.898362,2.795328e-04,4.064123e-03,10000,10000,10000,6.387299,2.425040e-04,3.547049e-03,8086.865583,8675.329593,8727.710426,-1913.134417,-1324.670407,-1272.289574
floki 2023-04-24 2023-05-24,floki,2023-04-24,2023-05-24,0.000045,1.619338e-09,2.417774e-08,10000,10000,10000,0.000032,1.201877e-09,1.757957e-08,7105.659301,7422.023952,7270.975063,-2894.340699,-2577.976048,-2729.024937
spool-dao-token 2023-05-05 2023-05-24,spool-dao-token,2023-05-05,2023-05-24,0.910145,3.083115e-05,4.565307e-04,10000,10000,10000,0.601966,2.285459e-05,3.342888e-04,6613.951842,7412.823884,7322.372669,-3386.048158,-2587.176116,-2677.627331
nakamoto-games 2023-05-19 2023-05-24,nakamoto-games,2023-05-19,2023-05-24,0.237491,8.833777e-06,1.310559e-04,10000,10000,10000,0.231264,8.780319e-06,1.284277e-04,9737.835873,9939.485189,9799.460337,-262.164127,-60.514811,-200.539663


In [22]:
realized_dt = realized.groupby('dt_out').agg({
    'usd_in': 'sum',
    'btc_in': 'sum',
    'eth_in': 'sum',
    'usd_out': 'sum',
    'btc_out': 'sum',
    'eth_out': 'sum'    
})

snapshots_dt = snapshots.groupby('dt_out').agg({
    'usd_in': 'sum',
    'btc_in': 'sum',
    'eth_in': 'sum',
    'usd_out': 'sum',
    'btc_out': 'sum',
    'eth_out': 'sum'    
})

In [23]:
realized_dt

,usd_in,btc_in,eth_in,usd_out,btc_out,eth_out
dt_out,,,,,,
2023-03-26,10000,10000,10000,9948.073403,9813.910559,9819.947827
2023-03-27,50000,50000,50000,45388.392822,46446.661245,46540.472038
2023-03-29,20000,20000,20000,20033.020242,19254.383172,19815.531071
2023-03-31,10000,10000,10000,9144.387934,8991.859005,8988.416531
2023-04-01,10000,10000,10000,9562.238901,9568.224781,9567.587175
2023-04-02,30000,30000,30000,29248.741016,28655.967210,28866.901340
2023-04-03,20000,20000,20000,18721.876064,18747.907848,18252.836573
2023-04-06,30000,30000,30000,28264.536986,28244.880658,27718.184976
2023-04-07,30000,30000,30000,25680.373603,25473.496790,25029.235872


In [25]:
#realized_dt.index = realized_dt.index.date.astype(str)
#realized_dt

In [26]:

realized_dt.loc['2023-03-28'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-03-30'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-04-04'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-04-05'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-04-13'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-04-15'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-04-16'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-04-17'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-04-18'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-04-19'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-04-21'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-04-29'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-05-02'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-05-05'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-05-07'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-05-09'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-05-10'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-05-11'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-05-12'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-05-13'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-05-14'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-05-15'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-05-16'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-05-17'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-05-18'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-05-20'] = [np.nan for i in range(len(realized_dt.columns))]
realized_dt.loc['2023-05-23'] = [np.nan for i in range(len(realized_dt.columns))]



realized_dt = realized_dt.sort_index()
realized_dt = realized_dt.fillna(value=0) #method='ffill')

In [27]:
realized_dt = realized_dt.expanding().sum()
realized_dt

,usd_in,btc_in,eth_in,usd_out,btc_out,eth_out
dt_out,,,,,,
2023-03-26,10000.0,10000.0,10000.0,9948.073403,9813.910559,9819.947827
2023-03-27,60000.0,60000.0,60000.0,55336.466226,56260.571804,56360.419865
2023-03-28,60000.0,60000.0,60000.0,55336.466226,56260.571804,56360.419865
2023-03-29,80000.0,80000.0,80000.0,75369.486468,75514.954976,76175.950936
2023-03-30,80000.0,80000.0,80000.0,75369.486468,75514.954976,76175.950936
2023-03-31,90000.0,90000.0,90000.0,84513.874403,84506.813981,85164.367467
2023-04-01,100000.0,100000.0,100000.0,94076.113304,94075.038762,94731.954641
2023-04-02,130000.0,130000.0,130000.0,123324.854320,122731.005972,123598.855981
2023-04-03,150000.0,150000.0,150000.0,142046.730384,141478.913820,141851.692554


In [28]:
def get_roi(df):
    df = deepcopy(df)
    df['roi_usd'] = (df['usd_out'] - df['usd_in']) / df['usd_in'] * 100
    df['roi_btc'] = (df['btc_out'] - df['btc_in']) / df['btc_in'] * 100
    df['roi_eth'] = (df['eth_out'] - df['eth_in']) / df['eth_in'] * 100
    return df

In [85]:
final_data = pd.DataFrame(columns = ['d'])

In [86]:
final_data.loc[0] = 1
final_data.loc[1] = np.nan

In [90]:
col = 'd'

q_05 = np.quantile(final_data[col].dropna(), q=0.05)
q_95 = np.quantile(final_data[col].dropna(), q=0.95)

final_data[f'q_05_{col}'] = np.nan
final_data[f'q_95_{col}'] = np.nan

final_data.loc[~final_data[col].isna(), f'q_05_{col}'] = q_05
final_data.loc[~final_data[col].isna(), f'q_95_{col}'] = q_95

In [91]:
final_data

,d,q_05_d,q_95_d
0,1,1.0,1.0
1,NaN,NaN,NaN


In [29]:
tog = pd.concat([snapshots_dt, realized_dt])
tog = tog.groupby(tog.index).sum()

get_roi(tog)

,usd_in,btc_in,eth_in,usd_out,btc_out,eth_out,roi_usd,roi_btc,roi_eth
dt_out,,,,,,,,,
2023-03-26,210000.0,210000.0,210000.0,221684.671899,218694.962752,218829.498324,5.564129,4.140458,4.204523
2023-03-27,240000.0,240000.0,240000.0,236084.004239,240378.441601,240888.519836,-1.631665,0.157684,0.370217
2023-03-28,250000.0,250000.0,250000.0,254600.649742,258237.766460,253067.882115,1.840260,3.295107,1.227153
2023-03-29,290000.0,290000.0,290000.0,305886.405628,299828.999411,301555.398921,5.478071,3.389310,3.984620
2023-03-30,290000.0,290000.0,290000.0,302632.350704,299437.873739,298689.816461,4.355983,3.254439,2.996488
2023-03-31,310000.0,310000.0,310000.0,327350.757725,319928.417550,319083.036393,5.597019,3.202715,2.930012
2023-04-01,320000.0,320000.0,320000.0,336324.088635,329089.184456,328219.936541,5.101278,2.840370,2.568730
2023-04-02,330000.0,330000.0,330000.0,342699.807818,337803.404689,338069.807235,3.848427,2.364668,2.445396
2023-04-03,330000.0,330000.0,330000.0,337338.740634,335736.340815,331388.118006,2.223861,1.738285,0.420642


In [44]:
snapshots['roi_btc'] = (snapshots['btc_out'] - snapshots['btc_in']) / snapshots['btc_in'] * 100
snapshots['roi_usd'] = (snapshots['usd_out'] - snapshots['usd_in']) / snapshots['usd_in'] * 100
snapshots['roi_eth'] = (snapshots['eth_out'] - snapshots['eth_in']) / snapshots['eth_in'] * 100

roi_btc = pd.pivot(snapshots, index='dt_out', columns='coingecko_id', values='roi_btc')
roi_eth = pd.pivot(snapshots, index='dt_out', columns='coingecko_id', values='roi_eth')
roi_usd = pd.pivot(snapshots, index='dt_out', columns='coingecko_id', values='roi_usd')

In [45]:
#add empty rows

In [46]:
realized['roi_btc'] = (realized['btc_out'] - realized['btc_in']) / realized['btc_in'] * 100
realized['roi_usd'] = (realized['usd_out'] - realized['usd_in']) / realized['usd_in'] * 100
realized['roi_eth'] = (realized['eth_out'] - realized['eth_in']) / realized['eth_in'] * 100

realized_btc = pd.pivot(realized, index='dt_out', columns='coingecko_id', values='roi_btc')
realized_usd = pd.pivot(realized, index='dt_out', columns='coingecko_id', values='roi_usd')
realized_eth = pd.pivot(realized, index='dt_out', columns='coingecko_id', values='roi_eth')


In [47]:
#input_p, main_config, cli_creds, cg_api_key = get_config('configs/config_5_25_03_2023/', 'configs/sensitive.yaml')
info = get_info(input_p)
info

,ticker,price_USDT,price_BTC,price_ETH,amount_usdt,dt
coingecko_id,,,,,,
3xcalibur,XCAL,0.247817,8.956111e-06,1.412884e-04,1,2023-03-25
aleph-zero,AZERO,1.464429,5.292441e-05,8.349164e-04,1,2023-03-25
alienfi,ALIEN,0.524992,1.897318e-05,2.993141e-04,1,2023-03-25
allianceblock-nexera,NXRA,0.070561,2.550076e-06,4.022908e-05,1,2023-03-25
aptos,APT,11.665619,4.215951e-04,6.650932e-03,1,2023-03-25
astar,ASTR,0.061291,2.215056e-06,3.494393e-05,1,2023-03-25
aura-finance,AURA,2.883135,1.041964e-04,1.643765e-03,1,2023-03-25
benddao,BEND,0.012567,4.541571e-07,7.164619e-06,1,2023-03-25
bone-shibaswap,BONE,1.121027,4.051388e-05,6.391324e-04,1,2023-03-25


In [43]:
def cg2ticker(metric_df, info):
    return metric_df.rename(columns=info['ticker'].to_dict())

roi_btc = cg2ticker(roi_btc, info)
roi_eth = cg2ticker(roi_usd, info)
roi_usd = cg2ticker(roi_usd, info)

realized_btc = cg2ticker(realized_btc, info)
realized_eth = cg2ticker(realized_usd, info)
realized_usd = cg2ticker(realized_usd, info)


NameError: name 'roi_btc' is not defined

In [44]:
gc = pygsheets.authorize(
    service_file=main_config['google-sheet']['path-to-key-file']
)
sh = gc.open_by_key(main_config['google-sheet']['sheet-id'])



In [45]:
info.index.name='coin'
write_df(info, sh, 'info', 1, 'days', to_UTC=True)

NameError: name 'info' is not defined

In [40]:

write_df(roi_usd, sh, 'roi_usd', 1, 'days', to_UTC=True)
write_df(roi_btc, sh, 'roi_btc', 1, 'days', to_UTC=True)
write_df(roi_eth, sh, 'roi_eth', 1, 'days', to_UTC=True)


#write_df(ndau_sum, sh, 'ndau_sum', 1, 'days', to_UTC=True)
#write_df(volume_sum, sh, 'volume_sum', 1, 'days', to_UTC=True)

In [41]:
write_df(realized_usd, sh, 'fixed_usd', 1, 'days', to_UTC=True)
write_df(realized_btc, sh, 'fixed_btc', 1, 'days', to_UTC=True)
write_df(realized_eth, sh, 'fixed_eth', 1, 'days', to_UTC=True)

In [30]:
prices_volumes = cli.query_dataframe(
    '''
    SELECT 
    coingecko_id, price, total_volume, date_of_candle_close as dt_candle_close
    FROM DEV_ODS_CG_MARKET.DEV_ALL_TOKEN_METRICS 
    where coingecko_id in {lst_coins}    
    '''.format(lst_coins = all_action_coins + ['bitcoin', 'ethereum'])

)
prices_volumes['dt_candle_close'] = prices_volumes['dt_candle_close'].astype(str)
prices_volumes = prices_volumes.drop_duplicates(subset=['coingecko_id', 'dt_candle_close'])

In [31]:
def iterator_window_df(df, first_date, window_length, last_date):
    window_end = datetime.strptime(first_date, '%Y-%m-%d')
    while window_end.strftime('%Y-%m-%d') <= last_date:
        window_start = window_end - timedelta(days=window_length)
        df_window = df[
            (df.index >= window_start.strftime('%Y-%m-%d')) &
            (df.index <= window_end.strftime('%Y-%m-%d'))
            ]

        yield window_end, df_window
        window_end += timedelta(days=1)

In [32]:
lst_coins = coins_today
first_enter = '2023-03-26'
period=30

In [33]:
prices_pivoted = prices_volumes.pivot(index='dt_candle_close', columns='coingecko_id', values='price')

In [34]:
base_coin = 'bitcoin'
df_beta_eth = pd.DataFrame(columns = lst_coins+[base_coin])
for dt, df_window in iterator_window_df(prices_pivoted, first_enter, period, dt_calc):
    daily_roi = (
        (df_window - df_window.shift(1)) / df_window.shift(1)
    ).iloc[1:]
    beta = daily_roi.cov()[base_coin] / (daily_roi.std()[base_coin] ** 2)
    df_beta_eth.loc[dt.strftime('%Y-%m-%d')] = beta

In [35]:
base_coin = 'ethereum'
df_beta_btc = pd.DataFrame(columns = lst_coins+[base_coin])
for dt, df_window in iterator_window_df(prices_pivoted, first_enter, period, dt_calc):
    daily_roi = (
        (df_window - df_window.shift(1)) / df_window.shift(1)
    ).iloc[1:]
    beta = daily_roi.cov()[base_coin] / (daily_roi.std()[base_coin] ** 2)
    df_beta_btc.loc[dt.strftime('%Y-%m-%d')] = beta

In [36]:
base_coin = 'ethereum'
df_corr_eth = pd.DataFrame(columns = lst_coins+[base_coin])
for dt, df_window in iterator_window_df(prices_pivoted, first_enter, 30, dt_calc):
    df_corr_eth.loc[dt.strftime('%Y-%m-%d')] = df_window.corr().loc[base_coin]

In [37]:
base_coin = 'bitcoin'
df_corr_btc = pd.DataFrame(columns = lst_coins+[base_coin])
for dt, df_window in iterator_window_df(prices_pivoted, first_enter, 30, dt_calc):
    df_corr_btc.loc[dt.strftime('%Y-%m-%d')] = df_window.corr().loc[base_coin]

In [38]:
for dt, df in portfolio_members.iloc[1:].iterrows():
    coin_in = set(portfolio_members.loc[dt, 'coins'])
    coin_not_in = set(df_corr_btc.columns).difference(set(coin_in)).difference({'bitcoin', 'ethereum'})
    df_corr_btc.loc[dt, coin_not_in] = np.nan
    df_beta_btc.loc[dt, coin_not_in] = np.nan
    df_corr_eth.loc[dt, coin_not_in] = np.nan
    df_beta_eth.loc[dt, coin_not_in] = np.nan
    

In [39]:
bc = pd.DataFrame(columns=['beta_btc', 'corr_btc', 'beta_eth', 'corr_eth'])

In [40]:
beta_btc = df_beta_btc.mean(axis=1).values.tolist()
corr_btc = df_corr_btc.mean(axis=1).values.tolist()
beta_eth = df_beta_eth.mean(axis=1).values.tolist()
corr_eth = df_corr_eth.mean(axis=1).values.tolist()

In [41]:
bc.loc[-1] = [
    beta_btc[-2],
    corr_btc[-2],
    beta_eth[-2],
    corr_eth[-2]
]


    

In [42]:
bc.loc[0] = [
    beta_btc[-1],
    corr_btc[-1],
    beta_eth[-1],
    corr_eth[-1]
]

In [44]:
bc

,beta_btc,corr_btc,beta_eth,corr_eth
-1,1.514459,0.699519,1.340054,0.667801
0,1.292527,0.684346,1.142075,0.647537


In [62]:
print('\n'.join([str(a) for a in df_beta_btc.mean(axis=1).values.tolist()]))

1.4026573105346578
1.236269267278457
1.346410006195958
1.2721850572970366
1.34323032707494
1.2903102682013652
1.2741919739528205
1.2540120019866212
1.3114978571896765
1.3453860361063026
1.3316468343860655
1.2825431402894412
1.227453860586396
1.2429471038779485
1.217368473036587
1.3441166333433558
1.223999308651243
1.2426232167215874
1.2752905430900656
1.3416490791568412
1.313788554036286
1.3506152269814489
1.40746355887406
1.3768754430124233
1.3528405866866857
1.2196077509086432
1.2684639068780323
1.3917626677091435
1.29191123774253
1.2579243505634654
1.2703402936137684
1.280805595723119
1.2779983966293769
1.230034589606084
1.2306666280759828
1.1928079920612698
1.1565499604498157
1.1419392255251615
1.1596324868009455
1.2079936593954068
1.1367627423311142
1.0160356063797287
1.0990515708649007
1.0725459560073107
1.0716339528221446
1.09345537462257
1.1372258873086942
1.1752935828349744


In [56]:
print('\n'.join([str(a) for a in df_beta_btc.mean(axis=1).values.tolist()]))

1.4026573105346578
1.236269267278457
1.346410006195958
1.2721850572970366
1.34323032707494
1.2903102682013652
1.2741919739528205
1.2540120019866212
1.3114978571896765
1.3453860361063026
1.3316468343860655
1.2825431402894412
1.227453860586396
1.2429471038779485
1.217368473036587
1.3441166333433558
1.223999308651243
1.2426232167215874
1.2752905430900656
1.3416490791568412
1.313788554036286
1.3506152269814489
1.3549437153087351


In [52]:
print('\n'.join([str(a) for a in df_beta_eth.mean(axis=1).values.tolist()]))

1.3873742109207667
1.2575958516707186
1.328444363321571
1.2660596404727746
1.310561956250697
1.257936606472587
1.2523176783319427
1.2516169370413281
1.2795146171902196
1.3032990805462925
1.288875363812605
1.2547656398381615
1.1696876324477252
1.235932964617578
1.1787854932259725
1.2378656296625383
1.1379002924093218
1.2913972473086248
1.3023619491821004


In [53]:
print('\n'.join([str(a) for a in df_corr_eth.mean(axis=1).values.tolist()]))

0.5553619978186476
0.553289225306066
0.5457276241215784
0.5402772586685113
0.5449048493173884
0.5254637438527349
0.4877388210777738
0.4781124218956506
0.5157632487380707
0.48503091838155327
0.4593451706526824
0.4947469496574107
0.479500057171909
0.5080600307259309
0.4886586959746572
0.35720770344857533
0.2846966371410395
0.25423921731228233
0.07478818963712094


In [54]:
print('\n'.join([str(a) for a in df_corr_btc.mean(axis=1).values.tolist()]))

0.5524881790207691
0.5502671672041239
0.5269886158745818
0.5262394233537327
0.5273981093992021
0.5007845581554646
0.46948096821854396
0.47061607046776477
0.4931104515273182
0.4607949872519187
0.4306303234933686
0.47222692665104776
0.47553141623650624
0.5226763219180303
0.4943747180749167
0.3725170727527159
0.3033708465179443
0.23792529518141242
0.09457872625977716


In [ ]:
def cg2ticker(metric_df, info):
    return metric_df.rename(columns=info['ticker'].to_dict())

roi_btc = cg2ticker(roi_btc, info)
roi_eth = cg2ticker(roi_usd, info)
roi_usd = cg2ticker(roi_usd, info)

realized_btc = cg2ticker(realized_btc, info)
realized_eth = cg2ticker(realized_usd, info)
realized_usd = cg2ticker(realized_usd, info)


In [60]:
for dt, df in portfolio_members.iterrows():
    vol_days = prices_volumes[prices_volumes['dt_candle_close'] == dt]
    vol_days = vol_days[vol_days['coingecko_id'].isin(df['coins'])]
    print(vol_days['total_volume'].mean())



12784362.098432938
20329179.138140503
23108354.838708855
30208218.454191048
23249367.365573674
39231466.89088985


In [165]:
vol_days

,coingecko_id,ndau,dt_candle_close


In [64]:
ndau['dt_candle_close'] = ndau['dt_candle_close'].astype(str)
for dt, df in portfolio_members.iterrows():
    vol_days = ndau[ndau['dt_candle_close'] == dt]
    vol_days = vol_days[vol_days['coingecko_id'].isin(df['coins'])]
    #print(dt)
    print(vol_days['ndau'].mean())



156.95
264.90909090909093
196.11111111111111
241.04545454545453
184.6
344.1363636363636


In [62]:
def query_get_coins_ndau_per_day(coins_1d, ndau_table):
    q = '''
        SELECT 
            coingecko_id, sum(ndau) as ndau, date_ndau as dt_candle_close
        FROM 
            {ndau_table}
        WHERE
            coingecko_id in {lst_coins}
        GROUP BY
            coingecko_id, date_ndau
        '''.format(
            lst_coins=tuple(coins_1d),
            ndau_table=ndau_table
        )
    return q

In [63]:
ndau = cli.query_dataframe(query_get_coins_ndau_per_day(coins_today, 'DEV_DM_ANALYTCS.NDAU_CROSS_BC'))

In [156]:
2.941468e+07

29414680.0

In [157]:
vol_days['total_volume'].mean()

12784362.098432938

In [158]:
vol_days['total_volume'].median()

3849935.709041893

In [100]:
portfolio_members['coins'].str.len()

2023-03-25    21
2023-03-26    23
2023-03-27    19
2023-03-28    23
2023-03-29    21
Name: coins, dtype: int64